We'll try to fetch the artists and tracks I listened to in a specific period from lastfm and see if we can encirh this info with spotify data (because it only allows to 50 last tracks in a beta api)
After that we can try som simple ML models on it :D 

In [2]:
import requests as re
#import numpy as np
import time
import pandas as pd
import datetime
from datetime import date, timedelta
from IPython.core.display import clear_output
import json
import requests_cache

### Pre-work

In [6]:
#Get the csv from the last request
#COLUMNS 0 -> BAND NAME
#COLUMNS 1 -> SONG NAME
#COLUMNS 2 -> PLAYED AT UNIX TIMESTAMP
tracks = pd.read_csv('played_tracks.csv')

In [7]:

tracks.drop(tracks.columns[2], axis = 1, inplace = True)
#clear duplicate columns
tracks = tracks.drop_duplicates()

## Spotify API

In [8]:
#create a request cache
requests_cache.install_cache('tags_for_tracks_cache')

In [9]:
#Authentication - getting an access token

auth_response = re.post('https://accounts.spotify.com/api/token', {
    'grant_type': 'client_credentials',
    'client_id': '6cb782c1b0404843b3e5a06e8361cb6e',
    'client_secret': 'a32c28deab714fa89091395089cbca90',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#create a header with the access token
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [12]:
tracks.iloc[0][0]

'Interpol'

In [14]:
#first request test
song_name = tracks.iloc[0][0]
band_name = tracks.iloc[0][1]
params = {'q' : song_name,
    'type': 'track'}

request = re.get('https://api.spotify.com/v1/search', headers = headers, params = params)

In [54]:
df = pd.json_normalize(request.json()['tracks'])

In [57]:
df.head()

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/search?query=Interp...,"[{'album': {'album_type': 'album', 'artists': ...",20,https://api.spotify.com/v1/search?query=Interp...,0,None,1039
